In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [3]:
tables = soup.find_all('table')
#[len(a) for a in a[0].find_all('tr')]

In [28]:
tmp = re.split('/| ', tables[0].find_all('caption')[0].get_text())
reporting_date = '-'.join(tmp[0:3])
print(reporting_date)

11-02-2020


In [29]:
# Get List of fields from Header Row
fieldList = []
fields = rows[0].find_all('th')
for field in fields:
    fieldList.append(field.get_text())
assert fieldList == ['FACILITY NAME', 'COUNTY','TYPE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS',
                     'NEW STAFF CASES', 'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']

In [37]:
# Create list of facilities
rows = tables[2].find_all('tr')
num_facilities = 0
facilities = []
first_row = True
for row in rows:
    if first_row:
        first_row = False
        continue
    facility = {}
    tds = row.find_all('td')
    num_reported_vals = 0
    facility['ReportingDate'] = reporting_date
    for i in range(len(tds)):
        val = tds[i].get_text()
        if val == '--':
            val = float('Nan')
        else:
            num_reported_vals = num_reported_vals +1
        facility[fieldList[i]] = val
    if num_reported_vals == 3:
        facility['Reporting'] = "No"
    else:
        facility['Reporting'] = "Yes"    
    facilities.append(facility)
print(len(facilities))
facilities[0]

1347


{'ReportingDate': '11-02-2020',
 'FACILITY NAME': 'Jamieson Nursing Home',
 'COUNTY': 'ALCONA',
 'TYPE': 'SNF',
 'NEW RESIDENT CASES': '0',
 'TOTAL RESIDENT CASES': '0',
 'NEW RESIDENT DEATHS': '0',
 'TOTAL RESIDENT DEATHS': '0',
 'NEW STAFF CASES': '0',
 'TOTAL STAFF CASES': '0',
 'NEW STAFF DEATHS': '0',
 'TOTAL STAFF DEATHS': '0',
 'Reporting': 'Yes'}

In [38]:
facilities_df = pd.DataFrame(facilities)

In [40]:
facilities_df.sort_values(by=['Reporting', 'NEW RESIDENT CASES'], ascending=False).head(20)

,ReportingDate,FACILITY NAME,COUNTY,TYPE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,TOTAL STAFF CASES,NEW STAFF DEATHS,TOTAL STAFF DEATHS,Reporting
138,11-02-2020,Medilodge of Marshall,CALHOUN,SNF,9,31,2,3,2,12,0,0,Yes
743,11-02-2020,Oakview Medical Care Facility,MASON,SNF,9,9,0,0,3,8,0,0,Yes
1301,11-02-2020,Rivergate Health Care Center,WAYNE,SNF,9,84,0,20,3,58,0,0,Yes
151,11-02-2020,Medilodge Of Cheboygan,CHEBOYGAN,SNF,8,8,0,0,3,6,0,0,Yes
185,11-02-2020,Christian Park HCC,DELTA,SNF,8,51,7,9,19,38,0,0,Yes
680,11-02-2020,Medilodge of Richmond,MACOMB,SNF,8,15,1,2,6,15,0,0,Yes
727,11-02-2020,Windemere Park Health and Rehab Center,MACOMB,SNF,8,18,1,1,3,13,0,0,Yes
941,11-02-2020,South Lyon Senior Care and Rehab,OAKLAND,SNF,8,12,0,0,1,6,0,0,Yes
1132,11-02-2020,Sanilac Medical Care Facility,SANILAC,SNF,8,9,0,0,4,5,0,0,Yes
419,11-02-2020,Faith Haven Senior Care Centre,JACKSON,SNF,7,7,0,0,3,4,0,0,Yes


In [42]:
facilities_df.sort_values(by=['Reporting', 'NEW RESIDENT CASES'], ascending=False).to_csv('Reporting_data/MI_' + reporting_date + 'LTC_data.csv', index=False)

In [44]:
facilites_reporting_cnt = len(facilities_df[facilities_df['Reporting'] == 'Yes'])
facilites_reporting_cnt

814

In [63]:
facilities_reporting_df = facilities_df[facilities_df['Reporting'] == 'Yes']
facilities_NOT_reporting_df = facilities_df[facilities_df['Reporting'] == 'No']

In [64]:
facilities_reporting_df['TYPE'].value_counts()

SNF    441
AFC    192
HFA    181
Name: TYPE, dtype: int64

In [65]:
facilities_NOT_reporting_df['TYPE'].value_counts()

AFC    412
HFA    120
SNF      1
Name: TYPE, dtype: int64